In [1]:
#for reproducability
SEED = 34

#maximum number of words in output text
MAX_LEN = 70

In [2]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

In [3]:
#get transformers
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# #get large GPT2 tokenizer and GPT2 model
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
# GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
#GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

#view model parameters
GPT2.summary()

Downloading: 100%|██████████| 1.04M/1.04M [00:11<00:00, 87.0kB/s]
Downloading: 100%|██████████| 456k/456k [00:05<00:00, 77.8kB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:08<00:00, 158kB/s]
Downloading: 100%|██████████| 665/665 [00:00<00:00, 444kB/s]
Downloading: 100%|██████████| 498M/498M [32:04<00:00, 259kB/s]
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Model: "tfgp_t2lm_head_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFGPT2MainLayer multiple                  124439808 
Total params: 124,439,808
Trainable params: 124,439,808
Non-trainable params: 0
_________________________________________________

In [4]:
import tensorflow as tf
tf.random.set_seed(SEED)

In [5]:
input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = GPT2.generate(input_ids, max_length = MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work. I want to get out of here. I want to get out of here. I want to get out of here. I want to get out of here. I want to get out of here. I want to get out of


In [6]:
# Beam search with N-Gram Penalities
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to get out of bed."

"I'm not going to lie to you," she said. "It's not like I've ever done anything like this before. It's just a matter of time."
1: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to get out of bed."

"I'm not going to lie to you," she said. "It's not like I've ever done anything like this before. It's just a matter of time before
2: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to get out of bed."

"I'm not going to lie to you," she said. "It's not like I've ever done anything like this before. It's just a matter of time.
3: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to get out of bed."

"I'm 

In [8]:
# Basic sampling
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 0, 
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work that I can't sit through all day. I need to get up and go to work!" Everyone must have heard that. People would have from school that sounded like a great idea. "Be like that, Joe. Someone is going


In [14]:
# Top-K sampling
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work!"

After five hours of questioning from the two assistants, they managed to find him and they finally decided to take him home. It turned out that when the train went over, Mr. Smith was nowhere to be found. ...


In [13]:
# Top-P sampling
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_p = 0.8, 
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work and being happy: eat like a champ. Why wouldn't I have done this? I'd be better off not worrying about anything else.

"If you don't want to eat well or even if you want to have long ...


In [15]:
# Top-K and Top-P Sampling
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50, 
                              top_p = 0.85, 
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work. To have your best and best friends to play with."

While most of the players who had been invited to watch the match were already signed to clubs for the season, he said they would join him for the end of the season.

He said: "I would like to play again next season, but only if I am selected.

"I want to go back to a place where I can train and do things that I enjoy.

"There are so many great players here. I don't know if I will come back to Tottenham or if...

1: I don't know about you, but there's only one thing I want to do after a long day of work and I'm about to go into labor."...

2: I don't know about you, but there's only one thing I want to do after a long day of work: get an hour of sleep every night. It doesn't happen that often. But I've heard about it a lot.

How much is 

In [16]:
!nvidia-smi

Wed Apr 14 13:45:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.72       Driver Version: 461.72       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   62C    P8    13W /  N/A |    653MiB /  6144MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
import torch

In [19]:
help(torch.bmm)

Help on built-in function bmm:

bmm(...)
    bmm(input, mat2, *, deterministic=False, out=None) -> Tensor
    
    Performs a batch matrix-matrix product of matrices stored in :attr:`input`
    and :attr:`mat2`.
    
    :attr:`input` and :attr:`mat2` must be 3-D tensors each containing
    the same number of matrices.
    
    If :attr:`input` is a :math:`(b \times n \times m)` tensor, :attr:`mat2` is a
    :math:`(b \times m \times p)` tensor, :attr:`out` will be a
    :math:`(b \times n \times p)` tensor.
    
    .. math::
        \text{out}_i = \text{input}_i \mathbin{@} \text{mat2}_i
    
    This operator supports :ref:`TensorFloat32<tf32_on_ampere>`.
    
    .. note:: This function does not :ref:`broadcast <broadcasting-semantics>`.
              For broadcasting matrix products, see :func:`torch.matmul`.
    
    Args:
        input (Tensor): the first batch of matrices to be multiplied
        mat2 (Tensor): the second batch of matrices to be multiplied
    
    Keyword Args

In [20]:
MAX_LEN = 150

In [21]:
prompt1 = 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

input_ids = tokenizer.encode(prompt1, return_tensors='tf')

In [22]:
# Benchmark Prompts
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

"We have been studying the language of humans for thousands of years. We discovered that this language, which we call Proto-Siberian, may be used in a variety of languages as well, as well as different cultures, including people of different ethnic backgrounds," said lead researcher Dr. Aja Jörg Jensen, from the University of Colorado and a member of the U.S. Geological Survey. "We're very excited to have found out about this discovery and that's a really exciting discovery."
...



In [23]:
# Fake news generated from GPT2
prompt2 = 'Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.'

input_ids = tokenizer.encode(prompt2, return_tensors='tf')

In [24]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.


The singer was spotted on a flight to Chicago to be photographed at a Target in St. Paul, Minnesota.


Scroll down for video


Savage: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today today


Ridiculous: Miley's mother told TMZ she heard her daughter's rant and then went to the Target and ordered the food

Miley Cyrus on the 'Glee' hit the set with a man in his 20s wearing a T-shirt and jeans with a 'Glee' logo at the Target in St. Paul today.


...



In [25]:
# Another fake news
prompt3 = 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.'

input_ids = tokenizer.encode(prompt3, return_tensors='tf')

In [26]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.

Abandoned by the orcs, he sent his minions across the vast wilderness. On his way, they reached a small rock formation and encountered a group of men. One of them made a run for it but they were unable to find it. A second, slightly larger orc led by Rhaegar attacked the group. He was easily defeated and Rhaegar returned to fight with his men, but was beaten by both of them.

In a fit of rage, he threw his armor over his head and cursed the men for being so foolish. When the men asked why they were not killed, he pointed at the orcs and...



In [27]:
# Fake homework assignemnts
prompt4 = "For today’s homework assignment, please describe the reasons for the US Civil War."

input_ids = tokenizer.encode(prompt4, return_tensors='tf')

In [28]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: For today’s homework assignment, please describe the reasons for the US Civil War.


1. The Civil War was a "civil war" that started on April 21st, 1865.


2. The US government tried to destroy the Confederacy by creating a new government, the Confederacy of Independent States. The new government was called the "Citizens of the United States" (or "DSA"), a title which would have meant all slaves were white. It was then that the "DSA" became known as the Confederacy of Independent States. The first African American president was named Abraham Lincoln.


3. In January of 1865, Confederate soldiers took the Civil War to the battlefield in the eastern part of the South, along...

